BIBLIOTECAS

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
import numpy as np
import unidecode

EXTRAÇÃO DOS DADOS

In [2]:
# Dados de acesso PostgreSQL
trt_name = 'automatiza_metricas_dev'
trt_user = 'automatiza'
trt_password = 'automatiza'
trt_host = '10.5.15.78'
trt_port = '5432'

# Acessa o banco de dados e cria a engine
db_url = f'postgresql+psycopg2://{trt_user}:{trt_password}@{trt_host}:{trt_port}/{trt_name}'
engine = create_engine(db_url)

# Seleciona as tabelas
query_cartoes = "SELECT * FROM cartoes"
query_checklists = "SELECT * FROM checklists"
query_listas = "SELECT * FROM listas"
query_quadros = "SELECT * FROM quadros"

# Armazena tabelas em dataframes
df_cartoes = pd.read_sql(query_cartoes, engine)
df_checklists = pd.read_sql(query_checklists, engine)
df_listas = pd.read_sql(query_listas, engine)
df_quadros = pd.read_sql(query_quadros, engine)

# Copia de segurança das tabelas
quadros = df_quadros.copy()
listas = df_listas.copy()
cartoes = df_cartoes.copy()
checklists = df_checklists.copy()

TRANSFORMAÇÃO DOS DADOS

In [3]:
# Renomear coluna "nome" das tabelas
quadros.rename(columns={'nome': 'quadro'}, inplace=True)
listas.rename(columns={'nome': 'lista'}, inplace=True)
cartoes.rename(columns={'nome': 'cartao'}, inplace=True)
checklists.rename(columns={'nome': 'checklist'}, inplace=True)

# Concatenar dataframes
consolidado = quadros.merge(listas, on='id_quadro', how='outer') \
    .merge(cartoes, on='id_lista', how='outer') \
        .merge(checklists, on='id_cartao', how='outer')

# Tratar valores nulos
consolidado['data_inicio'] = consolidado['data_inicio'].fillna(consolidado['data_fim'])
consolidado['data_fim'] = consolidado['data_fim'].fillna(consolidado['data_inicio'])
consolidado[['data_inicio', 'data_fim']] = consolidado[['data_inicio', 'data_fim']].fillna(pd.Timestamp('01-01-1900'))
consolidado['lead_time'] = consolidado['lead_time'].fillna(1)
consolidado['descricao'] = consolidado['descricao'].replace('', np.nan).fillna('sem descricao')
consolidado['id_lista'] = consolidado['id_lista'].replace('', np.nan).fillna('sem lista')
consolidado['lista'] = consolidado['lista'].replace('', np.nan).fillna('sem lista')
consolidado['id_cartao'] = consolidado['id_cartao'].replace('', np.nan).fillna('sem cartao')
consolidado['cartao'] = consolidado['cartao'].replace('', np.nan).fillna('sem cartao')
consolidado['id_checklist'] = consolidado['id_checklist'].replace('', np.nan).fillna('sem checklist')
consolidado['checklist'] = consolidado['checklist'].replace('', np.nan).fillna('sem checklist')
consolidado['quant_itens'] = consolidado['quant_itens'].replace('', np.nan).fillna(0)

# Padronizar itens em serie
def processar_texto(texto):
    texto = texto.lower() # letras minusculas
    texto = unidecode.unidecode(texto) # remove acentuação
    return texto
consolidado['lista'] = consolidado['lista'].apply(processar_texto)
consolidado['checklist'] = consolidado['checklist'].apply(processar_texto)

# Converter tipos de dados
consolidado['data_inicio'] = pd.to_datetime(consolidado['data_inicio'], format='%d-%m-%Y')
consolidado['data_fim'] = pd.to_datetime(consolidado['data_fim'], format='%d-%m-%Y')
consolidado['lead_time'] = consolidado['lead_time'].astype(int)
consolidado['quant_itens'] = consolidado['quant_itens'].astype(int)


In [4]:
display(consolidado.info())
#display(consolidado.head(2))
display(consolidado['quadro'].value_counts())
consolidado.to_csv('consolidado.csv', index=False)
consolidado.to_excel('consolidado.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 0 to 165
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id_quadro     166 non-null    object        
 1   quadro        166 non-null    object        
 2   id_lista      166 non-null    object        
 3   lista         166 non-null    object        
 4   id_cartao     166 non-null    object        
 5   cartao        166 non-null    object        
 6   descricao     166 non-null    object        
 7   data_inicio   166 non-null    datetime64[ns]
 8   data_fim      166 non-null    datetime64[ns]
 9   lead_time     166 non-null    int32         
 10  id_checklist  166 non-null    object        
 11  checklist     166 non-null    object        
 12  quant_itens   166 non-null    int32         
dtypes: datetime64[ns](2), int32(2), object(9)
memory usage: 15.7+ KB


None

quadro
Time SIP           87
SEDSJ              25
Testes Pyllo       17
Time Remessa       12
JT SIGEP            7
AutomatizaTRT       7
JTe                 6
Videoteca           1
Time Frequência     1
SIPAG               1
Projetos Legado     1
Núcleo WEB          1
Name: count, dtype: int64